In [12]:
#!pip install openpyxl

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import perceval as pcvl
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from merlin import LexGrouping, QuantumLayer, MeasurementStrategy, ComputationSpace
from merlin.builder import CircuitBuilder
import perceval as pcvl

import pandas as pd
import os

In [2]:
DATA_PATH = "./train.xlsx"

# Loading data

In [3]:
# Load training data (day-first dates)
df = pd.read_excel(DATA_PATH)
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df = df.sort_values('Date').reset_index(drop=True)

# Keep Date as first column
df = df[['Date'] + [c for c in df.columns if c != 'Date']]
feature_cols = [c for c in df.columns if c != 'Date']

print(f"Rows: {len(df)}")
print(f"Date range: {df['Date'].min().date()} → {df['Date'].max().date()}")
print(f"Number of surface points: {len(feature_cols)}")

df.head()

Rows: 494
Date range: 2050-01-01 → 2051-12-23
Number of surface points: 224


,Date,Tenor : 1; Maturity : 0.0833333333333333,Tenor : 2; Maturity : 0.0833333333333333,Tenor : 3; Maturity : 0.0833333333333333,Tenor : 4; Maturity : 0.0833333333333333,Tenor : 5; Maturity : 0.0833333333333333,Tenor : 6; Maturity : 0.0833333333333333,Tenor : 7; Maturity : 0.0833333333333333,Tenor : 8; Maturity : 0.0833333333333333,Tenor : 9; Maturity : 0.0833333333333333,...,Tenor : 5; Maturity : 30,Tenor : 6; Maturity : 30,Tenor : 7; Maturity : 30,Tenor : 8; Maturity : 30,Tenor : 9; Maturity : 30,Tenor : 10; Maturity : 30,Tenor : 15; Maturity : 30,Tenor : 20; Maturity : 30,Tenor : 25; Maturity : 30,Tenor : 30; Maturity : 30
0,2050-01-01,0.028565,0.038700,0.040127,0.040762,0.040466,0.038953,0.037553,0.036768,0.036646,...,0.331053,0.329056,0.330997,0.324676,0.325758,0.322393,0.345859,0.359162,0.346670,0.337670
1,2050-01-02,0.029334,0.039499,0.040982,0.041638,0.041336,0.039815,0.038397,0.037631,0.037504,...,0.336319,0.334434,0.336568,0.330244,0.331462,0.328144,0.351816,0.365197,0.350993,0.340822
2,2050-01-03,0.028696,0.038816,0.040328,0.041042,0.040804,0.039329,0.037968,0.037216,0.037107,...,0.333634,0.331707,0.333800,0.327487,0.328710,0.325436,0.348915,0.362236,0.348652,0.339027
3,2050-01-05,0.030854,0.041536,0.043035,0.043605,0.043241,0.041556,0.039977,0.039167,0.038968,...,0.336884,0.335106,0.337548,0.331279,0.332811,0.329771,0.353246,0.366100,0.351404,0.340465
4,2050-01-06,0.030406,0.041174,0.042681,0.043266,0.042937,0.041253,0.039685,0.038867,0.038667,...,0.333249,0.331426,0.333830,0.327580,0.329123,0.326146,0.349390,0.362048,0.348331,0.338022


In [4]:
class FinanceDataset(torch.utils.data.Dataset):
    """PyTorch Dataset for swaption volatility data."""
    
    def __init__(self, X, history=1, n_pred=1):
        self.X = torch.FloatTensor(X.values if isinstance(X, pd.DataFrame) else X)
        self.history = history
        self.n_pred = n_pred
    
    def __len__(self):
        return len(self.X) - self.history - self.n_pred
    
    def __getitem__(self, idx):
        return self.X[idx:idx+self.history], self.X[idx+self.history+1:idx+self.history+self.n_pred+1]

In [5]:
# Build a simple PyTorch Dataset using the next-day target
history=6

X = df[feature_cols].copy()
y = df[feature_cols].shift(-1)

# drop last row where target is NaN
mask = ~y.isnull().any(axis=1)
X = X[mask].reset_index(drop=True)

dataset = FinanceDataset(X, history=history, n_pred=3)
print(f"Dataset size: {len(dataset)} samples")
print(f"Input dim: {dataset[0][0].shape}, Target dim: {dataset[0][1].shape}")

input_dim = dataset[0][0].flatten().shape[0]
target_dim = dataset[0][1].flatten().shape[0]

Dataset size: 484 samples
Input dim: torch.Size([6, 224]), Target dim: torch.Size([3, 224])


In [6]:
from torch.utils.data import random_split, DataLoader

trainset, testset = random_split(dataset, [0.8, 0.2])

trainloader = DataLoader(trainset, batch_size=10, shuffle=True)
testloader = DataLoader(testset, batch_size=len(testset), shuffle=False) #no batches for testloader

# Testing

In [45]:
def test(model, loader, device="cpu", silent=True):
    model.eval()
    model.to(device)
    
    with torch.no_grad():
        inputs, targets = next(iter(loader))
        
        inputs = inputs.to(device).flatten(start_dim=1)
        targets = targets.to(device).flatten(start_dim=1)
        
        outputs = model(inputs)
        
        # --- R^2 Calculation ---
        # SS_res: Residual Sum of Squares (Variance of the error)
        sum_residual = ((targets - outputs)**2).sum(dim=0)
        
        # SS_tot: Total Sum of Squares (Variance of the data)
        target_mean = targets.mean(dim=0, keepdim=True)
        sum_square = ((targets - target_mean)**2).sum(dim=0)
        
        r_squared = 1 - sum_residual / (sum_square + 1e-8)
        
        mse = torch.nn.functional.mse_loss(outputs, targets, reduction='mean').item()
        root_mse = mse**0.5
        
        mae = torch.nn.functional.l1_loss(outputs, targets, reduction='mean').item()
        
    if not silent:
        print(f"Test Results:")
        print(f"\tMSE: {mse:.4f}")
        print(f"\tRoot MSE: {root_mse:.4f}")
        print(f"\tMAE: {mae:.4f}")
        print(f"\tR^2: {r_squared.mean().item():.4f}")
    
    return mse, root_mse, mae, r_squared

In [8]:
def example(model, loader):
    model.eval()
    
    with torch.no_grad():
        inputs, targets = next(iter(loader))
        outputs = model(inputs)
        
        relative_error = torch.abs(outputs - targets) / (torch.abs(targets) + 1e-7)
            
        print(f'Target={targets}, Output={outputs}, Rel_error={relative_error}')

# Training pipeline

In [13]:
def train(model, loss_fn, optim, n_epochs=10, device='cpu'):
    model.train()
    model.to(device)
    l = []

    pbar = tqdm(range(n_epochs))
    
    for i in pbar:
        cur_loss = 0
        cur_samples = 0
        
        for input, target in trainloader:
            input = input.to(device)
            target = target.flatten(start_dim=1).to(device)
            
            output = model(input)
    
            optim.zero_grad()
            loss = loss_fn(output, target)
            loss.backward()
            optim.step()
            
            cur_loss += loss.sum().cpu().item()
            cur_samples += input.shape[0]
            pbar.set_postfix(loss=cur_loss / cur_samples)
    
        #cur_loss /= len(trainloader)
        #pbar.set_postfix(prev_loss = cur_loss / cur_samples)

# The model

In [14]:
#Classical MLP
def classical_model(h_dim=1000):
    """
    Return a model and its actual number of
    parameters (proportional to `h_dim`)
    """
    model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_dim, h_dim),
        nn.ReLU(),
        nn.Linear(h_dim, target_dim)
    )
    return model

#print(f'Classical MLP size={sum(p.numel() for p in model.parameters() if p.requires_grad)}')

In [15]:
#Hybrid quantum model impl. with MerLin
def create_block(modes, step, max_param, univ_len):
    cir = pcvl.Circuit(modes)

    for i in range(modes):
        if i+step*modes >= max_param:
            break

        cir.add(i, pcvl.PS(pcvl.P(f'input{i+step*modes}')))

    rl = pcvl.GenericInterferometer(
        modes,
        lambda i: pcvl.BS()
        // pcvl.PS(pcvl.P(f"theta_ri{i+univ_len*step}"))
        // pcvl.BS()
        // pcvl.PS(pcvl.P(f"theta_ro{i+univ_len*step}")),
        shape=pcvl.InterferometerShape.RECTANGLE,
    )
    cir.add(0, rl)

    return cir
    

modes = 10
# left generic interferometer
wl = pcvl.GenericInterferometer(
    modes,
    lambda i: pcvl.BS()
    // pcvl.PS(pcvl.P(f"theta_li{i}"))
    // pcvl.BS()
    // pcvl.PS(pcvl.P(f"theta_lo{i}")),
    shape=pcvl.InterferometerShape.RECTANGLE,
)

circuit = pcvl.Circuit(modes)
circuit.add(0,wl)
for i in range(0, input_dim, modes):
    step = i // modes
    circuit.add(0, create_block(modes, step, input_dim, len(wl.params)))

quantum_layer = QuantumLayer(
    input_size=input_dim,
    circuit=circuit,
    n_photons=3,
    trainable_parameters=["theta"],
    input_parameters=["input"],
    dtype=dataset.X.dtype,
)

def quantum_model(h_dim=1000):
    model = nn.Sequential(
        nn.Flatten(),
        quantum_layer,
        LexGrouping(quantum_layer.output_size, 3*h_dim),
        nn.Linear(3*h_dim, target_dim)
    )
    return model

#print(f'Hybrid quantum model size={sum(p.numel() for p in model.parameters() if p.requires_grad)}')
#pcvl.pdisplay(circuit)

# Playground

In [47]:
from torch.optim import Adam

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = quantum_model(h_dim=100)
lr = 1e-3
loss_fn = nn.MSELoss()
optim = Adam(model.parameters(), lr=lr)

print(f'Model={model}')
print(f'Model size={sum(p.numel() for p in model.parameters() if p.requires_grad)}')

train(model, loss_fn, optim, device=device, n_epochs=100)

Model=Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): QuantumLayer(
    (_photon_loss_transform): PhotonLossTransform()
    (_detector_transform): DetectorTransform()
    (measurement_mapping): Probabilities()
  )
  (2): LexGrouping()
  (3): Linear(in_features=300, out_features=672, bias=True)
)
Model size=214512


  0%|          | 0/100 [00:43<?, ?it/s, loss=0.00583]


KeyboardInterrupt: 

In [27]:
torch.save(model.state_dict(), 'weights/classical_model_hist6.pt')

In [28]:
example(model, testloader)

Target=tensor([[0.0353, 0.0448, 0.0466,  ..., 0.4189, 0.3894, 0.3700],
        [0.0396, 0.0469, 0.0486,  ..., 0.4620, 0.4245, 0.3992],
        [0.0238, 0.0334, 0.0362,  ..., 0.3610, 0.3406, 0.3296],
        ...,
        [0.0339, 0.0446, 0.0474,  ..., 0.4194, 0.3853, 0.3642],
        [0.0232, 0.0330, 0.0361,  ..., 0.3522, 0.3328, 0.3224],
        [0.0254, 0.0368, 0.0402,  ..., 0.3531, 0.3330, 0.3212]]), Output=tensor([[0.0332, 0.0410, 0.0448,  ..., 0.4041, 0.3807, 0.3640],
        [0.0388, 0.0432, 0.0472,  ..., 0.4597, 0.4230, 0.4001],
        [0.0221, 0.0321, 0.0367,  ..., 0.3488, 0.3329, 0.3238],
        ...,
        [0.0317, 0.0422, 0.0449,  ..., 0.4163, 0.3856, 0.3679],
        [0.0233, 0.0331, 0.0378,  ..., 0.3408, 0.3222, 0.3139],
        [0.0244, 0.0358, 0.0406,  ..., 0.3443, 0.3239, 0.3140]]), Rel_error=tensor([[0.0609, 0.0829, 0.0383,  ..., 0.0354, 0.0224, 0.0162],
        [0.0183, 0.0778, 0.0292,  ..., 0.0049, 0.0036, 0.0023],
        [0.0725, 0.0374, 0.0136,  ..., 0.0337, 0.0

In [46]:
_ = test(model, testloader, silent=False)

Test Results:
	MSE: 0.0001
	Root MSE: 0.0107
	MAE: 0.0082
	R^2: 0.6909
